In [264]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Wool insulation

- Wool insulation around the gray bin inside the wooden box.  
- Two black tiles on top to keep the lid closed.  
- No bottles placed inside the gray bin.  
- No soil and worms

In [265]:
# Load the CSV
wool_insulation = pd.read_csv('data/wool_insulation.csv')

# Convert to time so it works with library functions
wool_insulation = wool_insulation.rename(columns={'last_changed': 'time'})

# Convert last_changed to datetime
wool_insulation['time'] = pd.to_datetime(wool_insulation['time'])

# Set time as index
wool_insulation = wool_insulation.set_index('time')

In [266]:
# Plot temperature data

# Map entity IDs to nicer names
entity_mapping = {
    'sensor.ds18b20_sensor_288ba6': 'Electronics',
    'sensor.ds18b20_sensor_28d206': 'Outside',
    'sensor.ds18b20_sensor_2805e1': 'Gray bin',
    'sensor.ds18b20_sensor_2855b9': 'Wooden box (outside wool)',
}

wool_insulation['entity_name'] = wool_insulation['entity_id'].map(entity_mapping)

# Plot using the nicer names
fig = px.line(
    wool_insulation,
    y='state',
    color='entity_name',
    title='WiggleBin with wool insulation'
)

# Hide Electronics and Outside by default
hidden_entities = ['Electronics', 'Outside']
for trace in fig.data:
    if trace.name in hidden_entities:
        trace.visible = 'legendonly'

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    legend_title='Sensor'
)

fig.show()

### Wool insulation and bottles of water

- Wool insulation around the gray bin inside the wooden box.  
- Two black tiles on top to keep the lid closed.  
- Two bottles with water placed inside the gray bin.  
- No soil and worms
- On the first day the bottles were added, there was a peak because the bottles were at room temperature.
  

In [267]:
# Load the CSV
wool_insulation_bottles = pd.read_csv('data/wool_insulation_and_bottles.csv')

# Convert to time so it works with library functions
wool_insulation_bottles = wool_insulation_bottles.rename(columns={'last_changed': 'time'})

# Convert last_changed to datetime
wool_insulation_bottles['time'] = pd.to_datetime(wool_insulation_bottles['time'])

# Set time as index
wool_insulation_bottles = wool_insulation_bottles.set_index('time')

# Map entity IDs to nicer names
entity_mapping = {
    'sensor.ds18b20_sensor_288ba6': 'Electronics',
    'sensor.ds18b20_sensor_28d206': 'Outside',
    'sensor.ds18b20_sensor_2805e1': 'Gray bin',
    'sensor.ds18b20_sensor_2855b9': 'Wooden box',
    'sensor.light_sensor': 'Light sensor'
}

wool_insulation_bottles['entity_name'] = wool_insulation_bottles['entity_id'].map(entity_mapping)

# Forward-fill on regular intervals
resampled_list = []

for entity, group in wool_insulation_bottles.groupby('entity_name'):
    group = group[['state']]  # select only numeric column
    group_resampled = group.resample('10min').ffill()  # 15-min interval, forward-fill
    group_resampled['entity_name'] = entity         # keep entity name
    resampled_list.append(group_resampled)

# Combine all entities
wool_insulation_bottles = pd.concat(resampled_list).reset_index()
wool_insulation_bottles = wool_insulation_bottles.set_index('time')

In [268]:
# Plot light sensor data
wool_insulation_bottles_light = wool_insulation_bottles[wool_insulation_bottles['entity_name'] == 'Light sensor']

# Plot using the nicer names
fig = px.line(
    wool_insulation_bottles_light,
    y='state',
    color='entity_name',
    title='WiggleBin light sensor data'
)

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Light Level (lux)',
    legend_title='Sensor'
)

fig.show()

In [269]:
# Plot temperature data

wool_insulation_bottles_temp = wool_insulation_bottles[wool_insulation_bottles['entity_name'] != 'Light sensor']

# Plot using the nicer names
fig = px.line(
    wool_insulation_bottles_temp,
    y='state',
    color='entity_name',
    title='WiggleBin with wool insulation and 2 bottles of water'
)

# Hide Electronics and Outside by default
hidden_entities = ['Electronics', 'Outside']
for trace in fig.data:
    if trace.name in hidden_entities:
        trace.visible = 'legendonly'

zone_colors = [
    {'y0': 10, 'y1': 15, 'color': 'lightblue', 'label': 'Low Productivity'},
    {'y0': 15, 'y1': 25, 'color': 'green', 'label': 'Peak Productivity'},
    {'y0': 25, 'y1': 30, 'color': 'orange', 'label': 'Low Productivity'},
]

# Add shaded zones
for zone in zone_colors:
    fig.add_shape(
        type="rect",
        x0=wool_insulation_bottles_temp.index[0], x1=wool_insulation_bottles_temp.index[-1],
        y0=zone['y0'], y1=zone['y1'],
        fillcolor=zone['color'],
        opacity=0.1,
        layer="below",
        line_width=0
    )

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    legend_title='Sensor'
)

fig.show()

In [270]:
from wiggle_bin_plot import plot_hours_in_temperature_zones

series_to_compare = {
    "Gray bin": wool_insulation_bottles_temp[wool_insulation_bottles_temp['entity_name'] == 'Gray bin']['state'],
    "Wooden box": wool_insulation_bottles_temp[wool_insulation_bottles_temp['entity_name'] == 'Wooden box']['state'],
}

fig = plot_hours_in_temperature_zones(series_to_compare, "WiggleBin - Time in Worm Temperature Zones")
fig.show()